In [1]:
import requests
import bs4
import pandas as pd
import re

### This is the notebook of our attempt at using webscraping to get the word counts of every COVID-19 related update from 10 schools
<p> Using pandas, beautiful soup, regex, and requests, we still struggled to make this method more efficient than simply 'copy pasting' each update into a word counter because of all the different HTML formats for the sites, and inconsisent and unreliable methods for counting the words on a page, and the sheer time it takes to code this for each site

#### *To jump to the conclusion, click [here](#anchor)*  



### Using Beautiful Soup, get the main site of updates and parse the links

In [2]:
rq = requests.get('https://chancellor.ucsb.edu/memos')
soup = bs4.BeautifulSoup(rq.text)

In [3]:
table_content = soup.find('table').find_all('tr')
table_content[0]

<tr>
<td class="views-field views-field-created views-align-left">May 14, 2020        </td>
<td class="views-field views-field-title views-align-left"><a href="/memos/2020-05-14-covid-19-update-planning-efforts-our-path-forward" hreflang="en">COVID-19 Update: Planning Efforts for Our Path Forward</a> </td>
</tr>

### Conveniently, the site used a table to store the links/dates, this will be used to convert it to and pandas df

In [4]:
dates = []
links = []
titles = []
for i in table_content:
    cont = i.find_all('td')
    dates.append(cont[0].text.strip())
    links.append(cont[1].find('a').attrs['href'])
    titles.append(cont[1].text)

In [5]:
assert(len(links) == len(dates))

### As you can see, the updates only go till Feb 7th, so we are only going to go through those

In [6]:
df = pd.DataFrame({'date':dates, 'title': titles, 'link':links})
df[:18]


,date,title,link
0,"May 14, 2020",COVID-19 Update: Planning Efforts for Our Path...,/memos/2020-05-14-covid-19-update-planning-eff...
1,"April 18, 2020",COVID-19 Update: Case at Nanofabrication Facil...,/memos/2020-04-18-covid-19-update-case-nanofab...
2,"April 13, 2020",COVID-19 Update: Summer Sessions,/memos/2020-04-13-covid-19-update-summer-sessions
3,"April 7, 2020",COVID-19 Update,/memos/2020-04-07-covid-19-update
4,"April 2, 2020",Message from President Napolitano and the UC C...,/memos/2020-04-02-message-president-napolitano...
5,"March 31, 2020",COVID-19 Update: Confirmed Cases in Our Commun...,/memos/2020-03-31-covid-19-update-confirmed-ca...
6,"March 26, 2020",COVID-19 Update: Commencement,/memos/2020-03-26-covid-19-update-commencement
7,"March 19, 2020",COVID-19 Update: Statewide Stay-at-Home Order,/memos/2020-03-19-covid-19-update-statewide-st...
8,"March 18, 2020",COVID-19 Update: Test Results and Library Serv...,/memos/2020-03-18-covid-19-update-test-results...
9,"March 17, 2020",COVID-19 Update: Test Result and Leave Policy,/memos/2020-03-17-covid-19-update-test-result-...


### Now I found the unique identifier for what makes up the body of the text...

In [7]:
df_covid = df[:17]
link = df_covid.iloc[1]['link']
orgin = 'https://chancellor.ucsb.edu/'
soup = bs4.BeautifulSoup(requests.get(orgin+link).text)

* But this is a major pain because I will have to parse every link, paragraph, and list element differently. 
* This is a problem because every page may have different formats too, and possibly different classes than listed here
* So I'm going to try and use regex to remove any tag

In [8]:
example = soup.find_all('div', attrs = {'property':"schema:text"})

In [9]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
cleaned = remove_html_tags(str(example))
cleaned[:750]

'[April 18, 2020\nDear Members of Our Campus Community,\nWe were informed today by a private company with access to our Nanofabrication facility in the Engineering-Sciences Building, that one of their employees who was in our facility on April 8 and 9, carrying out approved essential research, may have had COVID-19. The person did not have symptoms until April 10 and was tested several days later, and then informed their employer of testing positive. We wish them a fast, healthy recovery.\nThe company reported that the individual, who is not affiliated with UC Santa Barbara, was only in our Nanofabrication facility and has not been back to campus since April 9, which was prior to the onset of symptoms. We were also informed that the individual '

### This worked pretty well, now lets try to clear out those \n's

In [10]:
cleaned = cleaned.replace('\n', ' ')
cleaned[:750]

'[April 18, 2020 Dear Members of Our Campus Community, We were informed today by a private company with access to our Nanofabrication facility in the Engineering-Sciences Building, that one of their employees who was in our facility on April 8 and 9, carrying out approved essential research, may have had COVID-19. The person did not have symptoms until April 10 and was tested several days later, and then informed their employer of testing positive. We wish them a fast, healthy recovery. The company reported that the individual, who is not affiliated with UC Santa Barbara, was only in our Nanofabrication facility and has not been back to campus since April 9, which was prior to the onset of symptoms. We were also informed that the individual '

In [11]:
len(cleaned.split(' '))

498

### Looks good, now lets see if we can do this for all of our websites

In [12]:
links_list = df_covid['link']
orgin = 'https://chancellor.ucsb.edu/'

total_words = []
for link in links_list:
    soup = bs4.BeautifulSoup(requests.get(orgin+link).text)
    
    # finds the unique div that contains the body of text
    body = soup.find_all('div', attrs = {'property':"schema:text"})[0]
    
    # turns it into a sting and removes the tags and new line characters
    cleaned = remove_html_tags(str(body))
    cleaned = cleaned.replace('\n', ' ')
    total_words.append(len(cleaned.split(' ')))

In [13]:
print(total_words)

[1546, 498, 395, 484, 74, 1262, 603, 380, 452, 428, 284, 1338, 1226, 472, 896, 1052, 684]


### Lets throw it back into the previous table to see how it looks

In [14]:
df_covid = df_covid.assign(word_count = total_words)
df_covid.head(6)

,date,title,link,word_count
0,"May 14, 2020",COVID-19 Update: Planning Efforts for Our Path...,/memos/2020-05-14-covid-19-update-planning-eff...,1546
1,"April 18, 2020",COVID-19 Update: Case at Nanofabrication Facil...,/memos/2020-04-18-covid-19-update-case-nanofab...,498
2,"April 13, 2020",COVID-19 Update: Summer Sessions,/memos/2020-04-13-covid-19-update-summer-sessions,395
3,"April 7, 2020",COVID-19 Update,/memos/2020-04-07-covid-19-update,484
4,"April 2, 2020",Message from President Napolitano and the UC C...,/memos/2020-04-02-message-president-napolitano...,74
5,"March 31, 2020",COVID-19 Update: Confirmed Cases in Our Commun...,/memos/2020-03-31-covid-19-update-confirmed-ca...,1262


In [16]:
df_covid['date'] = pd.to_datetime(df_covid['date']).dt.strftime('%d-%b-%y')
output = df_covid[['date', 'word_count']].copy()
output['school'] = 'Santa Barbara'
output.to_csv('UC_Santa_Barbara.csv',index = False)

### It seemed to work seemlessly, but lets manually find the word count using a 'word counter' application to see how well it does

* for May 14th (the most recent), we calculated 1546 and the actual was 1542 *(diff of +4)*
* for March 19th (the median date), we calculated 380 and the actual was 377 *(diff of +3)*
* for February 7th (the first), we calculated 684 and the actual was 684 *(no difference)*

# <a name="anchor"></a>Conclusion

#### Pros:
* Can do all the sites in seconds when ran
* If the sites are formated simple enough, it *can* be fast
* Some sites *may* be similar enough to where we can just copy and paste a few variable names

#### Cons:
* Most of the sites we are analyzing aren't this simple. Some use layers of *Javascript* (such as Berkeley), which makes this whole process far more difficult
* This whole 'analyzing each school's HTML format' is different every time. There really aren't any consistancies aside from the methods we are using
* Overall not really any faster than just copying and pasting each update's content into a word counter and making CSV's by hand
* **And most importantly:** these results are prone to error, and in an incosistent amount
    * some *big* pages had *small* errors
    * meanwhile some *small* pages had *bigger* errors
    * unless we manually check each school for their error as well, we don't know how skewed they are either
        * and this is even more time consuming and really defeats the purpose of doing it automated, since we are doing a portion of it manually anyways